In [1]:
import os
import getpass
from pathlib import Path
from typing import Optional

from dotenv import load_dotenv  # type: ignore
from langchain_openai import ChatOpenAI  # type: ignore
from openai import OpenAI  # type: ignore


#* Exceção customizada para ausência da chave da API
class ApiKeyNotFoundError(Exception):
    """
    Exceção lançada quando a chave OPENAI_API_KEY não é encontrada no arquivo .env.
    Herda de Exception (herança simples em Python).
    Boas práticas: crie exceções específicas para facilitar o tratamento de erros.
    """
    pass

class ApiKeyLoader:
    """
    Utility to load the OpenAI API key from a .env file.
    If no path is provided, it searches for .env in the current and parent directories.
    """

    def __init__(self, env_path: Path | None = None) -> None:
        if env_path is not None:
            if not env_path.exists() or not env_path.is_file():
                raise ValueError(f"Invalid .env path: {env_path}")
            self.env_path = env_path
        else:
            found_env = self._find_env_path()
            if found_env is None:
                raise ValueError("Could not find a .env file in current or parent directories.")
            self.env_path = found_env

    def _find_env_path(self) -> Path | None:
        """Searches for a .env file in current and parent directories."""
        current = Path(__file__).resolve().parent
        for parent in [current, *current.parents]:
            candidate = parent / ".env"
            if candidate.exists() and candidate.is_file():
                return candidate
        return None

    def get_openai_key(self) -> str:
        load_dotenv(self.env_path)
        api_key = os.getenv("OPENAI_API_KEY")
        if not api_key:
            raise ApiKeyNotFoundError("OPENAI_API_KEY not found in .env file.")
        return api_key

In [2]:
class ChatModelFactory:
    """
    Factory class para criar diferentes tipos de modelos de chat do LangChain.

    Esta classe implementa o padrão Factory Method, permitindo a criação
    de diferentes configurações de modelos ChatOpenAI de forma centralizada
    e reutilizável.

    Args:
        api_key (str): Chave da API da OpenAI

    Example:
        >>> factory = ChatModelFactory("your-api-key")
        >>> analytical_model = factory.create_analytical_model()
        >>> creative_model = factory.create_creative_model()
    """

    def __init__(self, api_key: str) -> None:
        """
        Inicializa a factory com a chave da API.
        
        Args:
            api_key (str): Chave válida da API da OpenAI
            
        Raises:
            ValueError: Se a api_key estiver vazia ou None
        """
        if not api_key:
            msg = "OpenAI API key is required for ChatModelFactory"
            raise ValueError(msg)
        self.api_key: str = api_key

    def create_analytical_model(
        self, 
        model_name: str = "gpt-4", 
        temperature: float = 0.1
    ) -> ChatOpenAI:
        """
        Cria um modelo otimizado para análises e tarefas que requerem precisão.

        Configuração:
        - Temperatura baixa (0.1) para respostas mais determinísticas
        - Modelo GPT-4 por padrão para melhor capacidade analítica

        Args:
            model_name (str): Nome do modelo OpenAI (padrão: "gpt-4")
            temperature (float): Controla a aleatoriedade (padrão: 0.1)

        Returns:
            ChatOpenAI: Instância configurada para análises
        """
        return ChatOpenAI(
            api_key=self.api_key,  # type: ignore
            model=model_name,      # fixed
            temperature=temperature,
        )
    
    def create_creative_model(
        self, 
        model_name: str = "gpt-3.5-turbo", 
        temperature: float = 0.8
    ) -> ChatOpenAI:
        """
        Cria um modelo otimizado para tarefas criativas.
        
        Configuração:
        - Temperatura alta (0.8) para respostas mais criativas
        - GPT-3.5-turbo por padrão (mais rápido e econômico)
        
        Args:
            model_name (str): Nome do modelo OpenAI (padrão: "gpt-3.5-turbo")
            temperature (float): Controla a criatividade (padrão: 0.8)
            
        Returns:
            ChatOpenAI: Instância configurada para criatividade
        """
        return ChatOpenAI(
            api_key=self.api_key,  # type: ignore
            model=model_name,      # fixed
            temperature=temperature,
        
        )
    
    def create_conversational_model(
        self, 
        model_name: str = "gpt-3.5-turbo", 
        temperature: float = 0.7
    ) -> ChatOpenAI:
        """
        Cria um modelo balanceado para conversas naturais.
        
        Configuração:
        - Temperatura moderada (0.7) para equilíbrio entre precisão e naturalidade
        - GPT-3.5-turbo por padrão para boa performance
        
        Args:
            model_name (str): Nome do modelo OpenAI (padrão: "gpt-3.5-turbo")
            temperature (float): Controla a naturalidade (padrão: 0.7)
            
        Returns:
            ChatOpenAI: Instância configurada para conversas
        """
        return ChatOpenAI(
            api_key=self.api_key,  # type: ignore
            model=model_name,      # fixed
            temperature=temperature,
        )
    
    def create_custom_model(
        self,
        model_name: str,
        temperature: float,
        max_tokens: int = 2048,
        **kwargs: object
    ) -> ChatOpenAI:
        """
        Cria um modelo com configurações personalizadas.
        
        Args:
            model_name (str): Nome do modelo OpenAI
            temperature (float): Controla a aleatoriedade (0.0 a 2.0)
            max_tokens (int): Número máximo de tokens na resposta
            **kwargs: Argumentos adicionais para o modelo
            
        Returns:
            ChatOpenAI: Instância com configurações personalizadas
        """
        return ChatOpenAI(
            api_key=self.api_key,  # type: ignore
            model=model_name,      # fixed
            temperature=temperature,
        )

In [3]:
# === FUNÇÃO UTILITÁRIA PARA CRIAR MODELO ANALÍTICO ===
def create_analytical_model() -> Optional[ChatOpenAI]:
    """
    Função utilitária que demonstra o uso completo do módulo para criar
    um modelo analítico.

    Esta função:
    1. Carrega a chave da API do arquivo .env
    2. Cria uma instância da ChatModelFactory
    3. Retorna um modelo configurado para análises

    Returns:
        Optional[ChatOpenAI]: Modelo analítico ou None em caso de erro

    Example:
        >>> model = create_analytical_model()
        >>> if model:
        ...     response = model.invoke("Analise este texto...")
    """
    try:
        env_file_path: Path = Path(__file__).resolve().parent / ".env"
        loader: ApiKeyLoader = ApiKeyLoader(Path(env_file_path))
        openai_api_key = loader.get_openai_key()
        print(f"Chave da API OpenAI carregada com sucesso: ***{openai_api_key[-4:]}")

        chat_factory = ChatModelFactory(openai_api_key)
        analytical_llm = chat_factory.create_analytical_model()
        print("Modelo de chat OpenAI criado com sucesso!")
        return analytical_llm

    except (ValueError, ApiKeyNotFoundError) as e:
        print(f"Erro ao carregar a chave da API: {e}")
        return None

In [4]:
# # Uso avançado
loader = ApiKeyLoader(Path(".env"))
api_key = loader.get_openai_key()
factory = ChatModelFactory(api_key)
#* analytical model
analytical_model = factory.create_analytical_model()

In [5]:
from langchain.chat_models import init_chat_model
model = init_chat_model("gpt-4.1", model_provider="openai")


In [6]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Fabio")])

AIMessage(content='Hello Fabio! 😊 How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 11, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_b3f1157249', 'id': 'chatcmpl-BxCCaCz01FwzdDWTJw9OIeSsDDhpX', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c15e289e-7d59-4fe7-9118-7fe3f30aaa9c-0', usage_metadata={'input_tokens': 11, 'output_tokens': 11, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [7]:
from langchain_core.messages import AIMessage

response = model.invoke(
    [
        HumanMessage(content="Hi! I'm Fabio"),
        AIMessage(content="Hello Fabio! How can I assist you today?"),
        HumanMessage(content="What' my name?"),
    ]
)
print(response.content)  # Exibe a resposta do modelo

Your name is Fabio! How can I help you today, Fabio? 😊


In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

## Prompt Templates
___

Um Prompt Template é um modelo de texto usado para estruturar prompts (entradas de texto) enviados a um modelo de linguagem, como o GPT, de forma reutilizável e flexível. Ele permite que você insira variáveis dinamicamente no prompt, tornando-o mais organizado e fácil de manter.

No contexto do LangChain, o PromptTemplate ajuda a criar prompts com partes fixas e partes variáveis, de modo que o mesmo modelo de linguagem possa ser usado em diferentes situações apenas mudando os valores das variáveis.

In [9]:
from langchain.prompts import PromptTemplate

template = "Me fale 3 fatos sobre o uso de {tablets} para estudar e trabalhar."
prompt = PromptTemplate.from_template(template)

### Construtor Padrão


In [10]:
from langchain.prompts import PromptTemplate

template = "Me fale 3 fatos sobre o uso de {tablets} para estudar e trabalhar."
prompt = PromptTemplate(
    template=template,
    input_variables=["tablets"]
)

In [11]:
# Criar a mensagem formatada
formatted_prompt = prompt.format(tablets="samsung galaxy tab Fe10+")
print(formatted_prompt)  # Exibe o prompt formatado

Me fale 3 fatos sobre o uso de samsung galaxy tab Fe10+ para estudar e trabalhar.


In [12]:
model = analytical_model

In [13]:
response = model.invoke(formatted_prompt)
print(response.content)  # Exibe a resposta do modelo

1. Multitarefa: O Samsung Galaxy Tab S6 FE10+ é equipado com um processador poderoso que permite multitarefa sem esforço. Isso significa que você pode ter várias aplicações abertas ao mesmo tempo, como documentos de trabalho, e-mails, e até mesmo vídeos de estudo ou apresentações. Isso torna o tablet uma ferramenta eficiente para estudar e trabalhar, pois você pode alternar facilmente entre diferentes tarefas.

2. S Pen: Este tablet vem com a S Pen, que é uma caneta inteligente que permite que você faça anotações, desenhe e até mesmo controle o tablet à distância. Isso pode ser particularmente útil para estudar, pois você pode destacar informações importantes, fazer anotações à mão e até mesmo desenhar diagramas ou gráficos. Para o trabalho, a S Pen pode ser usada para assinar documentos digitalmente ou fazer apresentações.

3. Tela grande e de alta qualidade: O Samsung Galaxy Tab S6 FE10+ possui uma tela de 10,5 polegadas com resolução de 2560 x 1600 pixels. Isso proporciona uma exper

In [18]:
prompt = PromptTemplate(
    input_variables=["tablets"],
    template="Me fale 3 fatos sobre o uso de {tablets} para estudar e trabalhar."
)

In [19]:
model.invoke(formatted_prompt)

AIMessage(content='1. Multitarefa: O Samsung Galaxy Tab S6 FE10+ é equipado com um processador poderoso que permite multitarefa sem problemas. Isso significa que você pode ter várias aplicações abertas ao mesmo tempo, como um documento do Word, uma planilha do Excel e um navegador da web, sem que o tablet fique lento. Isso é especialmente útil para estudar e trabalhar, pois você pode alternar facilmente entre diferentes tarefas.\n\n2. S Pen: Este tablet vem com a S Pen, que é uma caneta stylus que permite escrever, desenhar e fazer anotações diretamente na tela. Isso pode ser muito útil para tomar notas durante uma aula ou reunião, ou para esboçar ideias durante um brainstorming. A S Pen também tem uma função de tradução, o que pode ser útil para estudar línguas estrangeiras.\n\n3. Tela grande e de alta qualidade: O Samsung Galaxy Tab S6 FE10+ tem uma tela de 10,5 polegadas com uma resolução de 2560 x 1600 pixels. Isso proporciona muito espaço para trabalhar e estudar, e a alta resoluç

In [20]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate

# Template para cada exemplo individual
example_prompt = PromptTemplate(
    input_variables=["operation", "step", "result", "explanation"],
    template="Operação: {operation}\nPasso: {step}\nResultado: {result}\nExplicação: {explanation}\n"
)

In [21]:
# Exemplos que mostram a progressão matemática
examples = [
    {
        "operation": "2 + 2",
        "step": "Soma simples",
        "result": "4",
        "explanation": "Adição básica: somamos 2 duas vezes"
    },
    {
        "operation": "2 × 2",
        "step": "Multiplicação como soma repetida",
        "result": "4",
        "explanation": "2 × 2 = 2 + 2 = 4 (multiplicação é soma repetida)"
    },
    {
        "operation": "2²",
        "step": "Exponenciação básica",
        "result": "4",
        "explanation": "2² = 2 × 2 = 4 (exponenciação é multiplicação repetida)"
    },
    {
        "operation": "2³",
        "step": "Expandindo a exponenciação",
        "result": "8",
        "explanation": "2³ = 2 × 2 × 2 = 8"
    },
    {
        "operation": "2⁴",
        "step": "Continuando o padrão",
        "result": "16",
        "explanation": "2⁴ = 2 × 2 × 2 × 2 = 16"
    }
]

In [22]:
# Prompt principal (suffix)
suffix = """
Agora resolva: {problem}
Siga o mesmo padrão dos exemplos acima.
Operação: {problem}
Passo:
Resultado:
Explicação:
"""

# Criando o Few-shot Prompt Template
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix=suffix,
    input_variables=["problem"]
)

# Testando com 2⁸
formatted_prompt = few_shot_prompt.format(problem="2⁸")
print(formatted_prompt)

# Usando com o modelo
response = model.invoke(formatted_prompt)
print("\n" + "="*50)
print("RESPOSTA DO MODELO:")
print(response.content) # type: ignore

Operação: 2 + 2
Passo: Soma simples
Resultado: 4
Explicação: Adição básica: somamos 2 duas vezes


Operação: 2 × 2
Passo: Multiplicação como soma repetida
Resultado: 4
Explicação: 2 × 2 = 2 + 2 = 4 (multiplicação é soma repetida)


Operação: 2²
Passo: Exponenciação básica
Resultado: 4
Explicação: 2² = 2 × 2 = 4 (exponenciação é multiplicação repetida)


Operação: 2³
Passo: Expandindo a exponenciação
Resultado: 8
Explicação: 2³ = 2 × 2 × 2 = 8


Operação: 2⁴
Passo: Continuando o padrão
Resultado: 16
Explicação: 2⁴ = 2 × 2 × 2 × 2 = 16



Agora resolva: 2⁸
Siga o mesmo padrão dos exemplos acima.
Operação: 2⁸
Passo:
Resultado:
Explicação:


RESPOSTA DO MODELO:
Passo: Continuando o padrão
Resultado: 256
Explicação: 2⁸ = 2 × 2 × 2 × 2 × 2 × 2 × 2 × 2 = 256


## Criar um chatbot
___

Neste desafio, você criará um chatbot que se lembra de interações anteriores, segue um fluxo de conversa estruturado e oferece respostas mais humanas usando o Few-Shot Prompting.
Ao aproveitar memória, prompts estruturados e exemplos few-shot, seu chatbot se comportará de forma consistente e envolvente.

**Cenário**
Você está desenvolvendo um assistente virtual para uma empresa. Seu chatbot precisa:
- Manter o histórico da conversa
- Responder de forma consistente usando exemplos predefinidos de few-shot
- Ser personalizável para diferentes estilos, como:
- Um assistente robótico com tom de ficção científica
- Um chatbot casual para interações divertidas
- Um assistente de IA profissional para tarefas empresariais
Ao final deste exercício, você terá um chatbot totalmente funcional que pode conversar de forma dinâmica, mantendo uma personalidade predefinida.

**Desafio**
Seu chatbot deve:
- Registrar o histórico da conversa
- Utilizar uma abordagem estruturada de Few-Shot Prompting
- Permitir a personalização de tom e personalidade


* 1. PERSONALIDADE (ChatPromptTemplate)
* 2. FORMATO/BLUEPRINT(FewShotPrompt)
* 3 ....

Pq até agora só vimos como inicializar o chatmodel, como estruturar a mensagem (SystemMessage, HumanMessage, AIMessage), Prompt Templates (basic PromptTemplate, Few-Shot PromptTemplate)

In [32]:
messages = [
    SystemMessage(content="You are a Python tutor"),
    HumanMessage(content="Explain what is a dictionary in Python")
]

In [34]:
ai_response = model.invoke(messages)
print(ai_response.content)  # Exibe a resposta do modelo

A dictionary in Python is a built-in data type that allows you to store key-value pairs. The keys in a dictionary are unique and can be of any immutable type (like integers, strings, tuples, etc.), while the values can be of any type. 

Dictionaries are mutable, which means you can add, remove, and change their key-value elements after they are created. They are also unordered, meaning the items do not have a defined order, and you cannot refer to an item by its index.

Here is an example of a dictionary:

```python
my_dict = {'name': 'John', 'age': 30, 'city': 'New York'}
```

In this dictionary, 'name', 'age', and 'city' are keys, and 'John', 30, and 'New York' are their respective values. You can access the values by their corresponding keys like this:

```python
print(my_dict['name'])  # Output: John
```

Dictionaries are very useful for data manipulation in Python, especially when dealing with large datasets.


In [36]:
topic = "Python classes"
prompt = f"Explain the concept of {topic} in simple terms."
ai_response = model.invoke(prompt)
print(ai_response.content)  # Exibe a resposta do modelo
print(f"Response for topic '{topic}': {ai_response.content}")

Python classes are the fundamental concept of object-oriented programming. A class is like a blueprint or template for creating objects (a particular data structure), providing initial values for state (member variables or attributes), and implementations of behavior (member functions or methods).

The class is a blueprint that defines the nature of a future object. From classes, we can construct instances. An instance is a specific object created from a particular class. For instance, if we have a class called "Car", we might have instances of that class like "Toyota", "Ford", etc. which are all cars but with different attributes and behaviors.

Classes encapsulate data and the methods that manipulate that data within one entity.
Response for topic 'Python classes': Python classes are the fundamental concept of object-oriented programming. A class is like a blueprint or template for creating objects (a particular data structure), providing initial values for state (member variables or

In [37]:
prompt_template = PromptTemplate(
    input_variables=["topic"],
    template="Explain the concept of {topic} in simple terms."
)

In [38]:
prompt_template

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Explain the concept of {topic} in simple terms.')

In [39]:
prompt_template.invoke({"topic": "Python classes"})

StringPromptValue(text='Explain the concept of Python classes in simple terms.')

In [41]:
model.invoke(
    prompt_template.invoke({"topic": "Python function"})  # type: ignore
)

AIMessage(content='A Python function is a block of reusable code that performs a specific task. Functions provide better modularity for your application and allow for high levels of code reusing. You can define functions to provide the required functionality. Here are simple rules to define a function in Python:\n\n- Function blocks begin with the keyword def followed by the function name and parentheses ().\n- Any input parameters or arguments should be placed within these parentheses. You can also define parameters inside these parentheses.\n- The first statement of a function can be an optional statement - the documentation string of the function or docstring.\n- The code block within every function starts with a colon (:) and is indented.\n- The statement return [expression] exits a function, optionally passing back an expression to the caller. A return statement with no arguments is the same as return None.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {

## Few Shot Prompt 
 - ensina padrão através de exemplos
* Uso: Treinar formato específico de resposta.

In [1]:
"""
Few-Shot Prompting com LangChain - Exemplo Didático
Demonstra como usar exemplos para treinar o modelo a seguir um padrão específico
"""

from langchain.prompts import FewShotPromptTemplate, PromptTemplate

# Exemplo 1: Few-Shot básico para classificação de sentimentos
def exemplo_classificacao_sentimentos():
    """
    Ensina o modelo a classificar sentimentos usando exemplos
    """
    
    # Exemplos que "ensinam" o modelo
    examples = [
        {
            "texto": "Eu amo este produto! É incrível!",
            "sentimento": "Positivo"
        },
        {
            "texto": "Este produto é terrível, não recomendo.",
            "sentimento": "Negativo"
        },
        {
            "texto": "O produto é ok, nada especial.",
            "sentimento": "Neutro"
        }
    ]
    
    # Template para cada exemplo
    example_prompt = PromptTemplate(
        input_variables=["texto", "sentimento"],
        template="Texto: {texto}\nSentimento: {sentimento}"
    )
    
    # Template Few-Shot completo
    few_shot_prompt = FewShotPromptTemplate(
        examples=examples,
        example_prompt=example_prompt,
        prefix="Classifique o sentimento dos seguintes textos:\n\n",
        suffix="Texto: {input}\nSentimento:",
        input_variables=["input"],
        example_separator="\n\n"
    )
    
    # Teste
    prompt = few_shot_prompt.format(input="Estou muito feliz com minha compra!")
    print("=== EXEMPLO 1: Classificação de Sentimentos ===")
    print(prompt)
    print("\n")

In [3]:
exemplo_classificacao_sentimentos()

=== EXEMPLO 1: Classificação de Sentimentos ===
Classifique o sentimento dos seguintes textos:



Texto: Eu amo este produto! É incrível!
Sentimento: Positivo

Texto: Este produto é terrível, não recomendo.
Sentimento: Negativo

Texto: O produto é ok, nada especial.
Sentimento: Neutro

Texto: Estou muito feliz com minha compra!
Sentimento:


